In [86]:
import pandas as pd
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk
nltk.download('vader_lexicon') 
from sklearn.metrics import confusion_matrix, accuracy_score

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
twitter = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv", sep=',')
amazon = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)
amazon = amazon.iloc[0:200,:]

In [75]:
print(list(twitter), list(amazon))

['Topic', 'Sentiment', 'TweetId', 'TweetDate', 'TweetText'] ['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime']


In [76]:
twitter = twitter[twitter['Sentiment'] != "irrelevant"]
twitter['Sentiment'].value_counts()

neutral     2333
negative     572
positive     519
Name: Sentiment, dtype: int64

In [0]:
def blob(stri):
  senttextblob = TextBlob(stri).sentiment
  poltextblob = senttextblob.polarity
  if poltextblob > 0.0:
    return "positive"
  elif poltextblob == 0.0:
    return "neutral"
  else:
    return "negative"

In [0]:
def nltk_(stri):
  s = SIA()
  sn = s.polarity_scores(stri)
  #return sn
  if sn['compound'] >= 0.05:
    return "positive"
  elif -0.05 < sn['compound'] < 0.05:
    return "neutral"
  else:
    return "negative"

In [80]:
twitter['sent_nltk'] = twitter['TweetText'].apply(nltk_)
twitter['sent_blob'] = twitter['TweetText'].apply(blob)
twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,sent_nltk,sent_blob
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,neutral,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,positive,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,positive,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,positive,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,positive,positive


In [92]:
amazon['sent_nltk'] = amazon['reviewText'].apply(nltk_)
amazon['sent_blob'] = amazon['reviewText'].apply(blob)
amazon.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sent_blob,sent_nltk
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004",positive,positive
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007",positive,positive
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011",positive,negative
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006",positive,positive
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013",positive,positive


In [87]:
nltk_matrix = confusion_matrix(twitter['Sentiment'], twitter['sent_nltk'])
blob_matrix = confusion_matrix(twitter['Sentiment'], twitter['sent_blob'])
nltk_acc = accuracy_score(twitter['Sentiment'], twitter['sent_nltk'])
blob_acc = accuracy_score(twitter['Sentiment'], twitter['sent_blob'])
print("NLTK confusion matrix is: ")
print(nltk_matrix)
print("NLTK accuracy is: ", nltk_acc)
print(" ")
print("TextBlob confusion matrix is: ")
print(blob_matrix)
print("TextBlob accuracy is: ", blob_acc)

NLTK confusion matrix is: 
[[ 291  138  143]
 [ 302 1234  797]
 [  47   99  373]]
NLTK accuracy is:  0.554322429906542
 
TextBlob confusion matrix is: 
[[ 234  171  167]
 [ 201 1213  919]
 [  38  124  357]]
TextBlob accuracy is:  0.5268691588785047
